In [1]:
from datetime import datetime as dt
import plotly.express as px
import dash                                     
from dash import dcc, html, Input, Output
import requests, json
import pandas as pd

In [2]:
# get names of assets / получаем названия валют
assets = requests.get('http://api.coincap.io/v2/assets').json()
assets_id = []
assets_symbol = []
for i in assets['data']:
    assets_id.append(i['id'])
    assets_symbol.append(i['symbol'])
assets_name = dict(zip(assets_id, assets_symbol))

# create DataFrame to work with / создаем таблицу для работы
url = 'http://api.coincap.io/v2/assets/bitcoin/history?interval=d1' # let's choose bitcoin for main view / начнем с bitcoin
json = requests.get(url).json()
df = pd.DataFrame(json['data'])
# fix types / исправляем типы данных
df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%dT%H:%M:%S.%f").dt.date
df['date'] = pd.to_datetime(df['date'])
df['priceUsd'] = df['priceUsd'].astype('float')

In [3]:
# load style / загружаем стиль
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# create dashboard / отрисовываем дашборд
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
                html.Div([
                    html.Div([
                        html.Label('Select an asset'),
                        dcc.Dropdown(
                                    options=assets_name,
                                    value='bitcoin',
                                    multi=False,
                                    id='my_dropdown',
                        ),
                        html.Div([
                            html.Div([
                                html.Label('Date from'),
                                dcc.DatePickerSingle(
                                    min_date_allowed = df['date'].min(),
                                    max_date_allowed = df['date'].max(),
                                    date=df['date'].min(),
                                    display_format='DD MM YYYY',
                                    id='my_date_picker_range_start'
                                ),
                            ], className = 'six columns'),
                            html.Div([
                                html.Label('Date to'),
                                dcc.DatePickerSingle(
                                    min_date_allowed = df['date'].min(),
                                    max_date_allowed = df['date'].max(),
                                    date=df['date'].max(),
                                    display_format='DD MM YYYY',
                                    id='my_date_picker_range_end'
                                ),
                            ], className = 'six columns'),
                        ], className = 'row'),
                    ], className = 'four columns'),
                    html.Div([
                        dcc.Graph(id='fig')
                    ], className = 'eight columns'),
                ], className = 'row'), 
            ])

@app.callback(
    Output('fig', 'figure'),
    Input('my_date_picker_range_start', 'date'),
    Input('my_date_picker_range_end', 'date'),
    Input('my_dropdown', 'value')
)
def update_output(start_date, end_date, value):
    # fill DataFrame with choosen asset / заполняем таблицу выбранной валютой
    url = 'http://api.coincap.io/v2/assets/' + str(value) + '/history?interval=d1'
    json = requests.get(url).json()
    df = pd.DataFrame(json['data'])
    # fix types / исправляем типы данных
    df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%dT%H:%M:%S.%f").dt.date
    df['date'] = pd.to_datetime(df['date'])
    df['priceUsd'] = df['priceUsd'].astype('float')

    mask = (df['date'] > start_date) & (df['date'] <= end_date)
    data = df.loc[mask]
    fig = px.bar(data, x="date", y="priceUsd")
    
    fig.update_layout(
    xaxis=dict(showgrid=False),
    yaxis=dict(zeroline=False, gridcolor='white'),
    title=assets_name.get(value),
    paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor='rgb(233,233,233)')
    
    return fig

# run dashboard / запускаем
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
